In [ ]:
TELEGRAM_BOT_TOKEN = ''
TELEGRAM_CHAT_ID = ''

In [ ]:
!pip install -q sklearn
!pip install -q matplotlib

In [ ]:
#from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd
import os
import io
from online_vectorizers import OnlineTfidfVectorizer
import scipy
import csv
import matplotlib.pyplot as plt
import pickle
import time

In [ ]:
base_dir = 'dataset_parsed_new'

#primo file
filename = "00_total_no_stopwords.csv"
fname = os.path.join(base_dir, filename)
df = pd.read_csv(fname)

#classe custom di libreria che estende quello base di scikit-learn. Aggiunge il partial-refit
vectorizer = OnlineTfidfVectorizer()

#necessario per inizializzare il vocabolario del vectorizer per poi fare partial fitting
#attualmente refitta due volte sul primo documento ma dovrebbe essere idempotente, alla peggio si aggiunge una guardia nel for
vectorizer.fit(df["text"].apply(lambda x: np.str_(x)))


#fittiamo il dizionario del vectorizer con tutte le parole in tutti i file. Necessario un partial refit per non mettere
#tutti i 200 file in RAM. Prima passata.
def fit_vectorizer(corpus_path):
    corpus_path, dirs, files = next(os.walk(corpus_path))
    for f in files:
        print(f)
        try:
            data_path = os.sep.join([base_dir, f])
            document = pd.read_csv(data_path)
            vectorizer.partial_refit(document["text"].apply(lambda x: np.str_(x)))
            print("La dimensione del vocabolario è:" + str(len(vectorizer.vocabulary_)))
        except Exception as e:
            print(e)

In [ ]:
fit_vectorizer(base_dir)
print("La dimensione del vocabolario è:" + str(len(vectorizer.vocabulary_)))
pickle.dump(vectorizer, open("vectorizer_new.pickle", "wb"))

In [ ]:
#creiamo la matrice delle features andando a trasformare ogni file tramite il vectorizer. Necessaria questa seconda passata
#in quanto non è possibile farlo in parallelo a causa del vectorizer che è pronto solo alla fine della prima passata.

#Qua non è necessario un partial refit ma è necessario concatenare ogni matrice di ogni documento, in modo da crearne una totale
#La funzione per "concatenare" matrici (orizzontalmente come serve a noi in questo caso) è vstack. Utilizzo matrix come
#accumulatore e lo restituisco alla fine
def transform(corpus_path):
    corpus_path, dirs, files = next(os.walk(corpus_path))
    matrix = scipy.sparse.csr_matrix((0, len(vectorizer.vocabulary_)), dtype=None)
    for f in files:
        print(f)
        try:
            data_path = os.sep.join([base_dir, f])
            document = pd.read_csv(data_path)
            partial_matrix = vectorizer.transform(document["text"].apply(lambda x: np.str_(x)))
            print("La shape della matrice del documento attuale è: " + str(partial_matrix.shape))
            print("La shape della matrice totale prima è: " + str(matrix.shape))
            matrix = scipy.sparse.vstack([matrix,partial_matrix])
            print("La shape della matrice totale dopo l'unione è: " + str(matrix.shape))
        except Exception as e:
            print(e)
    return matrix

In [ ]:
features_matrix = transform(base_dir)
print("La dimensione della matrice di features è:" + str(features_matrix.shape))
scipy.sparse.save_npz('features_matrix_on_documents_new', features_matrix)

In [ ]:
def load_matrix(filename='features_matrix_on_documents_new.npz'):
    return scipy.sparse.load_npz(filename)
features_matrix = load_matrix()

In [ ]:
#Calcolo e plot dell'elbow method per capire il numero giusto di clusters. In ogni caso io farei con 5 ma facciamolo comunque.
wcss = []
for i in range(1, 10):
  print(f"{time.strftime('%H:%M:%S')} training {i} clusters kMeans")
  kmeans = KMeans(n_clusters = i, init='k-means++', random_state=42)
  kmeans.fit(features_matrix)
  wcss.append(kmeans.inertia_)
plt.plot(range(1, 10), wcss)
plt.title('The Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.savefig('elbow_method_new.png')
plt.show()

In [ ]:
def send_telegram_message(text):
    if TELEGRAM_BOT_TOKEN and TELEGRAM_CHAT_ID:
        !pip install -q python-telegram-bot
        from telegram import Bot
        bot = Bot(token=TELEGRAM_BOT_TOKEN)
        bot.send_message(text=text, chat_id=TELEGRAM_CHAT_ID)
send_telegram_message("Finito di trainare dataset completo")

In [ ]:
#DA SCOMMENTARE E RUNNARE DOPO AVER RUNNATO AL CELLA PRECEDENTE

true_k = 6 #da scegliere dopo aver runnato la cella precedente in base al grafico
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
model.fit(features_matrix)
pickle.dump(model, open(f"new_kmeans_{true_k}.pkl", "wb"))


In [ ]:
#DA RUNNARE PER FARE PREDICTIONS, SCOMMENTARE LE PRIME DUE RIGHE SE SI VUOLE CARICARE IL MODELLO E IL VECTORIZER

model = pickle.load(open(f"new_kmeans_{true_k}.pkl", "rb"))
vectorizer = pickle.load(open("vectorizer_new.pickle", "rb"))

#per stampare feature più utili dei centroidi/clusters
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
true_k = len(order_centroids)
terms = vectorizer.get_feature_names()
s = ""
for i in range(true_k):
 s += f"Cluster {i}:\n"
 for ind in order_centroids[i, :20]:
  s += f"{terms[ind]}\n"

with open(f"new_clusters_{true_k}.txt", "w") as f:
    f.write(s)
    



#PER FARE PREDICTIONS SUI DOCUMENTI NUOVI, INSERIRE IL DOCUMENTO ALLA RIGA 3
#print("\n")
#print("Prediction")
#prova = vectorizer.transform(#inserire un documento da testare)
#predicted = model.predict(prova)
#print(predicted)

In [ ]:
print(s)

In [ ]:
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.cluster import KMeans
from sklearn.datasets import load_iris
import pylab as pl
pca = TruncatedSVD(n_components=2).fit(features_matrix)
pca_2d = pca.transform(features_matrix)
#pl.figure('Reference Plot')
# pl.scatter(pca_2d[:, 0], pca_2d[:, 1], c=iris.target)


In [ ]:
true_k = 6
kmeans = pickle.load(open(f"new_kmeans_{true_k}.pkl", "rb"))
pl.figure(f'K-means with {true_k} clusters')
pl.scatter(pca_2d[:, 0], pca_2d[:, 1], c=kmeans.labels_)
pl.savefig(f'k-means with {true_k} clusters.png')
pl.show()

In [ ]:
legend = []
for i in range(true_k):
    legend.append('Cluster {0}'.format(i))
pl.figure(f'K-means with {true_k} clusters')
pl.scatter(pca_2d[:, 0], pca_2d[:, 1], c=kmeans.labels_)
pl.legend()
pl.savefig(f'k-means with {true_k} clusters.png')
pl.show()

In [ ]:
send_telegram_msg("Finito di salvare scatter plot")